# Task 2: A Sample of Owners

It would be highly efficient to have a local sample of owners to do work. Here, we will generate a file that contains ever record for each owner. In order to accomplish this we will set up a python script below that will carry out the following tasks:

1. Connects to my Google Big Query instance.

2. Builds a list of owners. 

3. Takes a sample of the owners. 

4. Extracts all records associated with the sample of owners and writes them to a local text file. 

In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
import zipfile
from datetime import datetime
from pandas_gbq import *
from pandas_gbq import to_gbq
from pandas_gbq import read_gbq
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
# Set Up Google Big Query Instance
service_path = "C:/Users/breni/Documents/"
service_file = 'niekampbreannawedge-8bbebeea1dda.json'
project_id = 'niekampbreannawedge'
data_id = 'wedge24'

beans_key = service_path + service_file

credentials = service_account.Credentials.from_service_account_file(beans_key)

client = bigquery.Client(credentials= credentials, project= project_id)


In [5]:
# Build a List of Owners
query_owners = """
    SELECT DISTINCT card_no
    FROM `umt-msba.wedge_transactions.transArchive*`
    WHERE card_no != 3
"""
df_owners = client.query(query_owners).to_dataframe()


Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/niekampbreannawedge/jobs?prettyPrint=false: Access Denied: Project niekampbreannawedge: User does not have bigquery.jobs.create permission in project niekampbreannawedge.

Location: None
Job ID: 7da2e740-7f1d-4832-8898-f7dbefac93f0


In [ ]:
# Take Sample of the Owners

In [ ]:
# Extract All Records Associated with the Sample of Owners, 
# and writes them to a local text file. 
